# Particle Sand

Using penalty-based collisions to simulate particle sand. 
When a collision is detected, the model adds a virtual spring of rest length 0 between the colliding objects

Author: Andres Ibarra, written as part of 22F Leave Term Research

Credit to Professor Bo Zhu, on whose COSC89 course notes the collision logic is based

In [8]:
import taichi as ti

ti.init()

max_particles = 100
num_particles = ti.field(dtype=ti.i32, shape=())
dt = 1e-3
substeps = 10
screen_size = 800
bowl_r = 400
c = ti.Vector([0.5, 0.75]) # center of the bowl

ks = 1e2
kd = 0.5e1

x = ti.Vector.field(2, dtype=ti.f32, shape=max_particles)
v = ti.Vector.field(2, dtype=ti.f32, shape=max_particles)
f = ti.Vector.field(2, dtype=ti.f32, shape=max_particles)
r = ti.field(dtype=ti.f32, shape=max_particles)
m = ti.field(dtype=ti.f32, shape=max_particles)

@ti.func
def calculate_forces():
    """Calculates body and collision forces for all particles."""
    for i in range(num_particles[None]):
        f[i] = [0,-2] # reset force to be solely gravity
        # boundary check with the bowl
        phi = bowl_r - ((x[i] - c).norm() * screen_size) - r[i]
        neg_grad = (x[i] - c).normalized()
        if phi < 0: # outside of the bowl
            f[i] += ks * (phi) * neg_grad
            f[i] += kd * (ti.Vector([0, 0]) - v[i]).dot(neg_grad) * neg_grad
            
        for j in range(i+1, num_particles[None]):
            # check for collision
            phi = (x[i] - x[j]).norm() - r[i]/screen_size - r[j]/screen_size
            normal = (x[j] - x[i]).normalized()
            if phi < 0:
                force_spring = ks * (phi) * normal
                force_damp = kd * (v[j] - v[i]).dot(normal) * normal
                f[i] += force_spring
                f[j] -= force_spring
                f[i] += force_damp
                f[j] -= force_damp

@ti.func
def update_values():
    """Updates velocity and position for all particles."""
    for i in range(num_particles[None]):
        v[i] = v[i] + dt * f[i] / m[i]
        x[i] = x[i] + dt * v[i]

@ti.kernel
def substep():
    calculate_forces()
    update_values()
        
@ti.kernel
def new_particle(pos_x: ti.f32, pos_y: ti.f32):
    particle_id = num_particles[None]
    if particle_id < max_particles:
        x[particle_id] = [pos_x, pos_y]
        v[particle_id] = [0, 0]
        m[particle_id] = 1
        r[particle_id] = 10
        num_particles[None] += 1
        

def main():
    gui = ti.GUI('Particle Sand', (screen_size, screen_size))
    
    new_particle(0.3, 0.5) 
    
    while gui.running:
        for e in gui.get_events(ti.GUI.PRESS):
            if e.key == ti.GUI.UP:
                ks *= 1.1
            elif e.key == ti.GUI.DOWN:
                ks /= 1.1
            elif e.key == ti.GUI.LMB:
                new_particle(e.pos[0], e.pos[1])
            
        # Move stuff
        for step in range(substeps):
            substep()
             
        # Draw particles
        X = x.to_numpy()
        for i in range(num_particles[None]):
            gui.circle(pos=X[i], radius = r[i])

        # show the current values
        gui.text(
            content='Click anywhere on screen to add a new mass',
            pos=(0,0.99),
            color=0xffffff)
        
        gui.show()
    
if __name__ == '__main__':
    main()

[Taichi] Starting on arch=x64
